In [2]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [3]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### Volume Herding

In [4]:
query = '''
    SELECT
        CusipId,
        TrdExctnDt,
        ABS(BuyAmount - SellAmount) / (BuyAmount + SellAmount) AS Hm,
        CASE
            WHEN BuyAmount > SellAmount THEN 'Buy Herding'
            WHEN SellAmount > BuyAmount THEN 'Sell Herding'
        END AS Flag
    FROM (
        SELECT
            CusipId,
            TrdExctnDt,
            SUM(CASE WHEN RptSideCd = 'S' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS BuyAmount,
            SUM(CASE WHEN RptSideCd = 'B' THEN EntrdVolQt * RptdPr / 100 ELSE 0 END) AS SellAmount
        FROM
            Trace_filteredWithRatings
        WHERE
            CntraMpId = 'C'
        GROUP BY
            CusipId,
            TrdExctnDt
    ) A
    WHERE
        BuyAmount <> 0 AND SellAmount <> 0
    ORDER BY
        TrdExctnDt,
        CusipId
'''

# read sql
df = pd.read_sql(query, connection)
df.to_csv('source/volumeHerding.csv', index=False)